# Statistics 101
In this notebook, we will discuss some basic concepts in statistics that you need for your assignment.

You already know the concept of a mean, being the sum of the items divided by the number of items.

we are using numpy for this. numpy is a library that can do array manipulation in a very fast way. All the things you will do later is based on numpy.

In [ ]:
import numpy as np
X = np.array([1,3,7,5,3,8,9])
length = X.shape[0]
print(X.sum()/length)
print(X.mean())

As you see, you can calculate the mean by taking sum and divide it by the number of elements in the array. You can also directly take the mean by using `X.mean()` 

The mean itself does not say that much. if you have a set of numbers, uniformly ranging from 0 to 1000, the mean (or average) will be around 500, but this average does not say that much about the individual values. For that reason, we try to capture the spread of set of numbers. We can go into the details of normal distributions and Gauss, but we won't do that for now. We just state how we measure this spread: We take the difference from each individual cases with the calculated mean:

In [ ]:
differences = X - X.mean()
print(differences)

if we would add up these values, they would end up at zero (or a very very small value):

In [ ]:
print(differences.sum())

if we take the sum of squares of these differences, the signs disapear in the individual values and we get something of a total spread: the sum of squares:

In [ ]:
differences_squared = differences ** 2
print(differences_squared)
print(f"Sum of squares: {differences_squared.sum()}")

This value is actually not the one we like, since it grows with the number of items, so we divide it by the number of items, giving us the so called *variance*:

In [ ]:
variance = differences_squared.sum() / differences_squared.shape[0]
print(f"variance: {variance}")

We can calculate the variance of a numpy array with `X.var()`

In [ ]:
print(f"variance: {X.var()}")

The *standard deviation* is the square root of the variance:

In [ ]:
print(f"{X.std()} = {np.sqrt(X.var())}")

This number tells us something about how sure we are about an estimate, like the mean. There is a theory about it (the one of Gauss with his normal distribution), which we are not going to explain now. I will, for now explain it by an example. Suppose we take the daily temperatures in the summer for the region of south limburg (maastricht). For this, open the file `temps.csv`: 

In [ ]:
import csv
with open("temps.csv", "rt") as f:
    reader = csv.reader(f)
    temps = np.array(list(reader)).astype(float)
print(temps)

The first column (column 0 in python) is the day number in July, the second is the average temperature, the third is the maximum temperature and the last is the minimum temperature. We can take the daily average temperature by selecting the second column:

In [ ]:
T = temps[:,1]
print(T)

## Assignment 1.
Please calculate the average, variance and standard deviation of this array

In [ ]:
# insert your code here

As you see from the series, the temperatures fluctuate and are most of the time under 20 degrees. For a statistician it could be interesting to know if the temperature is *significantly* lower than 20 degrees. in that case, it is not only enough to look at the average, but also the standard deviation. Rule of thumb within statistics (with a 95% certainty) is to create an area of about two standard deviations below and two standard deviations about the mean and see if the 20 degrees falls within this interval

In [ ]:
interval = (T.mean() - 2*T.std(), T.mean()+2*T.std())
print(interval)

as you can see, the interval goes from 13.6 degrees to 22.6 degrees. the temperature does not deviate significantly from 20 degrees. All the temperatures in this range are reasonable temperatures during this period. Question is: what does the average say about the temperature?

We call this the confidence interval, and it introduces an uncertainty on the value we are measuring. The role of statistics is to see what we still can claim about the temperature in july around maastricht.

Scientific findings often only have their validity with respect to this error and that is why it is important to look at them. Even everything you read about whatever statistics is valid under some assumptions, where the confidence interval is one of. 

# Population

Another important concept we talk about is population. What are we making a statistics about? all dutch people? all dutch inhabitants? All dutch companies? note that the word *dutch* already restricts the population. As Statistics Netherlands, we make statistics about something that is Dutch. Not because we are nationalistic, but because we have to deliver information to the other parts of the government so they can make policies. 

So the concept population is important. If we say that the average length of a dutch male is 1.84 and of a dutch female is 1.70, this means that we have to estimate this length as accurate as possible for all individuals in the dutch population: this statement should be as true as possible.

So, how do we do that? First, let us generate a dutch population. The lengths have a standard deviation of about 7 cm, so we can create a population based on this information. You don't have to understand this completely, but here is the code.

In [ ]:
from numpy.random import normal, shuffle, choice
avgmale = 184
sdmale = 7
avgfemale = 170
sdfemale = 7
Nmale = 8745468
Nfemale = 8845004

males = normal(avgmale, sdmale, Nmale)
print(f"males: {males.mean()} +/- {males.std()}")
females = normal(avgfemale, sdfemale, Nfemale)
print(f"females: {females.mean()} +/- {females.std()}")
population = np.concatenate([males,females])
shuffle(population)
print(f"The average length of dutch people is {population.mean()} cm, with a standard deviation of {population.std()}")

#for the future, we save this population
males = np.concatenate([males, np.zeros(males.shape)]).reshape(2,Nmale).transpose()
females = np.concatenate([females, np.ones(females.shape)]).reshape(2,Nfemale).transpose()
dataset = np.concatenate([males,females])
shuffle(dataset)
import csv
with open("lengths.csv", "wt") as f:
    writer = csv.writer(f)
    writer.writerows(dataset)

You see that the total standard deviation is larger than the standard deviation per gender. Why is that?

Now we will determine the average length of a dutch person by what we call a simple random sample. We take a sample of 100 people and determine on the basis of them what the average is:

In [ ]:
sample = choice(population, replace=False, size=100)
print(sample.mean())

you see that, everytime you execute the previous cell, another value occurs, but it is always around that 177cm that we found in the population. 

This fluctuation depends on the number of samples that we take. The more samples we take, the less the value fluctuates. In the next cell, a simple script is written that takes the average of 100 repetitions of this experiment and calculates the variance. note that we use the variable `sample_size` to determine how many samples we take.

In [ ]:
sample_size = 100
estimated_lengths = []
for rep in range(100):
    sample = choice(population, replace=False, size=sample_size)
    estimated_lengths.append(sample.mean())
print(f"variance with a sample size of 100 is equal to: {np.array(estimated_lengths).var()}")

With list comprehension, we can write this shorter:

In [ ]:
sample_size = 100
estimated_lengths = np.array([choice(population, replace=False,size=sample_size).mean() for _ in range(100)])
variance = estimated_lengths.var()

Now we want to do this for different sample sizes. We will not do that for many sample sizes but for some. this script takes a while, so start it and take a break.

In [ ]:
from tqdm import tqdm
variances = []
SAMPLE_SIZES = [50,100,200,400,800,1600]
for sample_size in tqdm(SAMPLE_SIZES):
    estimated_lengths = np.array([choice(population, replace=False,size=sample_size).mean() for _ in range(100)])
    variances.append(estimated_lengths.var())
    

Now we plot the results and see something really interesting: The bigger the sample size is, the smaller the variance is between the individual estimates. And further: Already with 800 samples, the value is quite stable.

In [ ]:
import matplotlib.pyplot as plt
variances = np.array(variances)
sample_sizes = np.array(SAMPLE_SIZES)
plt.plot(sample_sizes, variances)

This is in a nutshell what we do when we send out questionaires: we optimize the number of samples in such a way that we can say, with a certain confidence, that, for instance, the average length of a dutch citizen is X cm. 

The double loop, programmed above is exactly what is going to happen in the next notebook, where we try to determine the confidence of the estimate of a certain variable based on the amount of data we put inside a machine learning algorithm. Have fun!!!